In [1]:
import networkx as nx
import itertools

def is_left_prime(generator_matrix):
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    R = generator_matrix[0, 0].parent()
    snf_gen = generator_matrix.smith_form()[0]
    snf_wanted = block_matrix([[identity_matrix(R, k), zero_matrix(R, k, n-k)]])
    return snf_gen == snf_wanted

def row_degrees(generator_matrix):
    row_degs = []
    n_rows = generator_matrix.nrows()
    # calculate the row degrees.
    for i in range(n_rows):
        row_deg = max([polynomial.degree() for polynomial in generator_matrix[i]])
        row_degs.append(row_deg)
    return row_degs

def is_row_reduced(generator_matrix):
    R = generator_matrix[0, 0].parent()
    x = R.0
    field = R.base()
    n = generator_matrix.ncols()
    k = generator_matrix.nrows()
    row_degs = row_degrees(generator_matrix)
    h_r_coeff_mat_list = []
    for i in range(k):
        h_r_coeff = matrix(field, (generator_matrix[i] - (generator_matrix[i] % x^row_degs[i])) / x^row_degs[i])
        h_r_coeff_mat_list.append(h_r_coeff)
    h_r_coeff_mat = block_matrix(k, 1, h_r_coeff_mat_list)
    return h_r_coeff_mat.rank() == k

def get_states(generator_matrix):
    # we assume the generator matrix is left prime and row reduced.
    # otherwise an error is raised.
    row_degs = row_degrees(generator_matrix)
    R = generator_matrix[0,0].parent()
    field = R.base()
    q = field.cardinality()
    #if not is_left_prime(generator_matrix):
        #raise ValueError("Generator Matrix is not left prime")
    n_rows = generator_matrix.nrows()
    #if not is_row_reduced(generator_matrix):
        #raise ValueError("Generator Matrix is not row reduced")
    # write down the possible states for each row
    states_to_be_combined = [
        [tuple(list(elt)) for elt in list(VectorSpace(field, row_degs[i]))] for i in range(n_rows)
    ]
    # write down all possible states by considering all combinations of the states of the rows.
    states = itertools.product(*states_to_be_combined)
    return row_degs, list(states)

In [2]:
def dec_to_base_q(number, q=2):
    n = number
    if n == 0:
        return [0]
    base_q_exp = []
    while n != 0:
        base_q_exp.append(n % q)
        n = n//q
    return base_q_exp

def dec_to_base_q_padded(number, length, q=2):
    # computes the binary representation of a decimal number in the way we need it
    # for the registers
    str_list = dec_to_base_q(number, q)
    bin_list = str_list + [0 for _ in range(length-len(str_list))]
    return matrix(1, length, bin_list)

def coefficients_up_to_deg(f, n):
    # returns the coefficients of a polynomial up to the coefficient of degree n.
    field = f.parent().base()
    coeff_list = f.coefficients(sparse=False)
    coeff_list = coeff_list + [0 for _ in range(n - len(coeff_list))]
    return matrix(field, n, 1, coeff_list)

In [3]:
#TODO: check if all the generalizations to F_q are okay.
# calculate the output of a row, given state and input, i.e., of a rate 1/n generator matrix
# where we possibly add zeros depending on the memory of the initial generator matrix.
def generate_output_row(row, state_row, row_input, memory):
    n = len(row)
    g_i_list = [coefficients_up_to_deg(f, memory + 1) for f in row]
    G_M_n = block_matrix(1, n, g_i_list, subdivide=False)
    field = G_M_n[0, 0].parent()
    q = field.cardinality()
    input_state_vector = vector(field,
                                [row_input] + list(state_row) + [0 for _ in range(memory - len(state_row))]
    )
    input_state_matrix = matrix(1, memory + 1, input_state_vector)
    return input_state_matrix * G_M_n

In [4]:
# calculate the output of a generator matrix given a state and input
# do this by adding the outputs of each row.
def generate_output(generator_matrix, state, inp, row_degrees):
    field = generator_matrix[0,0].parent().base()
    memory = max(row_degrees)
    n_rows = generator_matrix.nrows()
    output_list = []
    for i in range(n_rows):
        output_row = generate_output_row(generator_matrix[i], state[i], inp[i], memory)
        output_list.append(output_row)
    output = zero_matrix(field, 1, generator_matrix.ncols())
    # add up all the outputs from the rows.
    for i in range(n_rows):
        output += output_list[i]
    return output

In [5]:
R.<x> = PolynomialRing(GF(2), 'x')
generator_matrix = matrix(R, [[1, x^2 + 1, x^2 + x + 1]])
row_degs, states = get_states(generator_matrix)
generator_matrix = matrix(R, [[1, x^2 + 1, x], [x^2 + x + 1, 1, x^3 + 1]])
row_degs, states = get_states(generator_matrix)

In [6]:
def next_state_row(generator_matrix, state, row_degree, input_data):
    R = generator_matrix[0].parent()
    field = R.base()
    q = field.cardinality()
    if len(list(state)) > 0:
        next_state = tuple([input_data] + list(state)[:-1])
    else:
        next_state = ()
    #print("Next State Row", next_state)
    return next_state

def next_state(generator_matrix, state, row_degrees, input_data):
    R = generator_matrix[0,0].parent()
    field = R.base()
    q = field.cardinality()
    len_input = len(state)
    next_state = []
    inp = list(input_data)
    output = generate_output(generator_matrix, state, inp, row_degrees)
    # for each "part", i.e., each row of the input compute the corresponding part/row of the next state.
    for i in range(len_input):
        next_state_r = next_state_row(generator_matrix[i], state[i], row_degrees[i], input_data[i])
        next_state.append(next_state_r)
    return tuple(next_state), output

def possibilities_inputs(generator_matrix, row_degrees):
    R = generator_matrix[0,0].parent()
    field = R.base()
    q = field.cardinality()
    len_input = len(row_degrees)
    inputs_to_be_combined = [[elt for elt in list(field)] for _ in range(len_input)]
    possible_inputs = list(itertools.product(*inputs_to_be_combined))
    return possible_inputs
    
# calculate the possible next states in a state transition diagram given the states we have.
def next_states(generator_matrix, state, row_degrees):
    #print("State", state)
    len_input = len(state)
    possible_inputs = possibilities_inputs(generator_matrix, row_degrees)
    next_states = []
    # generate the next state for each possible input
    for i in range(len(possible_inputs)):
        inp = possible_inputs[i]
        new_state, output = next_state(generator_matrix, state, row_degrees, inp)
        # append the next state and the corresponding output.
        next_states.append((inp, new_state, output))
    #print("Nest States", next_states)
    return next_states

In [7]:
def previous_state_row(generator_matrix, state, row_degree, input_data):
    R = generator_matrix[0].parent()
    field = R.base()
    q = field.cardinality()
    if len(list(state)) > 0:
        previous_state = tuple(list(state)[1:] + [input_data])
    else:
        previous_state = ()
    return previous_state

def previous_state(generator_matrix, state, row_degrees, input_data):
    # same as for next state, only input is considered to be from the right and without output
    R = generator_matrix[0,0].parent()
    field = R.base()
    q = field.cardinality()
    len_input = len(state)
    previous_state = []
    inp = list(input_data)
    # for each "part", i.e., each row of the input compute the corresponding part/row of the next state.
    for i in range(len_input):
        previous_state_r = previous_state_row(generator_matrix[i], state[i], row_degrees[i], input_data[i])
        previous_state.append(previous_state_r)
    return tuple(previous_state)
                             
def previous_states(generator_matrix, state, row_degrees):
    len_input = len(state)
    possible_inputs = possibilities_inputs(generator_matrix, row_degrees)
    previous_states = []
    # generate the previous state for each possible input
    for i in range(len(possible_inputs)):
        inp = possible_inputs[i]
        prev_state = previous_state(generator_matrix, state, row_degrees, inp)
        # append the next state and the corresponding output.
        previous_states.append((inp, prev_state))
    return previous_states

In [8]:
def make_diagram(generator_matrix, states, row_degrees):
    diagram = nx.MultiDiGraph()
    diagram.add_nodes_from(states)
    # for each state add the edges in the diagram.
    new_states = {}
    old_states = {}
    for i in range(len(states)):
        next_states_i = next_states(generator_matrix, states[i], row_degrees)
        #print("state i", states[i])
        #print("Next States i", next_states_i)
        # one edge consists of the current state, next state and the output corresponding to the edge.
        edges_from_i = [
            (tuple(states[i]),
             tuple(next_state_i[1]),
             {"output" : next_state_i[2]}) for next_state_i in next_states_i
        ]
        diagram.add_edges_from(edges_from_i)
        #print("Edges", diagram.edges())
    backward_diagram = diagram.reverse()
    #print(backward_diagram.nodes())
    zero_state = tuple([tuple([0 for _ in range(row_degrees[i])]) for i in range(len(row_degrees))])
    next_states2 = backward_diagram[zero_state]
    #print("Next States 2", next_states2)
    for i in range(len(states)):
        next_states_i = next_states(generator_matrix, states[i], row_degrees)
        new_states[states[i]] = {}
        old_states[states[i]] = {}
        new_states[states[i]]["next state"] = {}
        old_states[states[i]]["previous state"] = {}
        for next_state_i in next_states_i:
            # next_state_i[0] is the input
            #diagram.nodes[states[i]]["next state"][next_state_i[0]] = next_state_i[1]
            new_states[states[i]]["next state"][next_state_i[0]] = next_state_i[1]
        previous_states_i = previous_states(generator_matrix, states[i], row_degrees)
        for previous_state_i in previous_states_i:
            #diagram.nodes[states[i]]["previous state"][previous_state_i[0]] = previous_state_i[1]
            old_states[states[i]]["previous state"][previous_state_i[0]] = previous_state_i[1]
    nx.set_node_attributes(diagram, new_states)
    nx.set_node_attributes(diagram, old_states)
    return diagram, backward_diagram

#diagram, backward_diagram = make_diagram(generator_matrix, states, row_degs)
#edges = list(diagram.edges())
# to access the edge we need a additional edges[0] because we use a multidigraph.

In [9]:
#print(diagram.nodes(data=True))
#print(diagram[((0, 0), (0, 0, 0))][((0, 0),(0, 0, 0))]) # what is happening here?